In [1]:
# -*- coding:utf-8 -*-
import openpyxl
from bs4 import BeautifulSoup
from urllib.request import urlopen
import csv, re, requests
import time

# 엑셀파일 열기
filename = "test.xlsx"
book = openpyxl.load_workbook(filename)

# 맨 앞의 시트 추출하기
sheet = book.worksheets[0]

# 시트의 각 행을 순서대로 추출하기
data = []
for row in sheet.rows:
    data.append([row[0].value, row[1].value,row[2].value])

del data[0]

for i in range(len(data)):
    print(data[i])

print(len(data))

[' DM500S9Z-AD3BA', 'SAMSUNG', '2020.11.19']
[' FA506IU-HN174', 'ASUS', '2020.11.19']
[' GA502IV-AZ001T', 'ASUS', '2020.11.19']
[' NT750QCJ-K58A', 'SAMSUNG', '2020.11.19']
[' NT950XCJ-X716A', 'SAMSUNG', '2020.11.19']
['15M A10SE-i7', 'MSI', '2020.11.19']
['15U50N-KR56K', 'LG', '2020.11.19']
['15UD50N-GX50K', 'LG', '2020.11.19']
['15ZD90N-VX5BK', 'LG', '2020.11.19']
['17UD70N-G.AX76K', 'LG', '2020.11.19']
['800G5 TWR 6BD60AV', 'HP', '2020.11.19']
['AS G512LI-HN065', 'ASUS', '2020.11.19']
['B80FV', 'LG', '2020.11.19']
['DB400T9A i5-9400', 'SAMSUNG', '2020.11.19']
['FA706IU-H7100', 'ASUS', '2020.11.19']
['G512LU-AL043', 'ASUS', '2020.11.19']
['G712LU-EV001', 'ASUS', '2020.11.19']
['G731GT-H7114', 'ASUS', '2020.11.19']
['GA502IU-AZ015T', 'ASUS', '2020.11.19']
['GA502IV-AZ001', 'ASUS', '2020.11.19']
['GF75 Thin 9SCXR', 'MSI', '2020.11.19']
['GP75 Leopard 9SD', 'MSI', '2020.11.19']
['GU502LV-AZ015', 'ASUS', '2020.11.19']
['GU502LW-HC111T', 'ASUS', '2020.11.19']
['GX550LWS-HC013R', 'ASUS', '2

# Beautiful 빠르나 결국 URL당 접근하면 느려지기때문에 selenium 검색과 시간 비교

In [2]:
start = time.time()  # 시작 시간 저장
result=[]


for i in range(len(data)):
    if data[i][0]== None:
        break
    else:
        url = 'https://search.shopping.naver.com/search/all?query=' + data[i][0]
        print(url)
        req = requests.get(url)   # 크롤링할 페이지 url로 요청 보내기 , req를 출력하면 statuscode만 출력됨
        html = req.text                # html 코드를 가져온다!
        soup = BeautifulSoup(html, 'html.parser')
        if soup.find('span', class_="price_num__2WUXn").text == None:
                         result.append("해당상품없음")
        else:
            data[i].append(soup.find('span', class_="price_num__2WUXn").text)
            
# 작업 코드
 
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
print(data)

https://search.shopping.naver.com/search/all?query= DM500S9Z-AD3BA
https://search.shopping.naver.com/search/all?query= FA506IU-HN174
https://search.shopping.naver.com/search/all?query= GA502IV-AZ001T
https://search.shopping.naver.com/search/all?query= NT750QCJ-K58A
https://search.shopping.naver.com/search/all?query= NT950XCJ-X716A
https://search.shopping.naver.com/search/all?query=15M A10SE-i7
https://search.shopping.naver.com/search/all?query=15U50N-KR56K
https://search.shopping.naver.com/search/all?query=15UD50N-GX50K
https://search.shopping.naver.com/search/all?query=15ZD90N-VX5BK
https://search.shopping.naver.com/search/all?query=17UD70N-G.AX76K
https://search.shopping.naver.com/search/all?query=800G5 TWR 6BD60AV
https://search.shopping.naver.com/search/all?query=AS G512LI-HN065
https://search.shopping.naver.com/search/all?query=B80FV
https://search.shopping.naver.com/search/all?query=DB400T9A i5-9400
https://search.shopping.naver.com/search/all?query=FA706IU-H7100
https://search.s

# DB에 해당 데이터 뿌려주기 

2. MSSQL 사용 절차,
Python에서 MSSQL에 있는 데이타를 사용하는 일반적인 절차는 다음과 같다.<br>

pymssql 모듈을 import 한다<br>
pymssql.connect() 메소드를 사용하여 MSSQL에 Connect 한다. 호스트명, 로그인, 암호, 접속할 DB 등을 파라미터로 지정할 수 있다.<br>
DB 접속이 성공하면, Connection 객체로부터 cursor() 메서드를 호출하여 Cursor 객체를 가져온다. DB 커서는 Fetch 동작을 관리하는데 사용된다.<br>
Cursor 객체의 execute() 메서드를 사용하여 SQL 문장을 DB 서버에 보낸다.<br>
SQL 쿼리의 경우 Cursor 객체의 fetchall(), fetchone(), fetchmany() 등의 메서드를 사용하여 데이타를 서버로부터 가져온 후, Fetch 된 데이타를 사용한다.<br>
삽입, 갱신, 삭제 등의 DML(Data Manipulation Language) 문장을 실행하는 경우, INSERT/UPDATE/DELETE 후 Connection 객체의 commit() 메서드를 사용하여 데이타를 확정 갱신한다.<br>
Connection 객체의 close() 메서드를 사용하여 DB 연결을 닫는다.<br>
아래 예제들의 기본 샘플 데이타로 아래와 같은 Customer 테이블이 있다고 가정하자.<br>

In [3]:
print(data[0][3])
print(type(data[0][3]))

548,730원
<class 'str'>


UTF-8 문제로인하여 해결되지않았던 문제를 해결함. <br>
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbf in position 11: invalid start byte <br>
위에 나온 문제가 계속나와서 별것도아닌걸로 계속 끙끙됌 

In [4]:
import datetime


now = datetime.datetime.now()
nowDate = now.strftime('%Y-%m-%d')
print(nowDate)

2020-11-23


In [9]:
 # -*- coding: utf-8 -*-

import easygui
import pymssql
import pandas as pd

now = datetime.datetime.now()
nowDate = now.strftime('%Y-%m-%d')
print(nowDate)

# MSSQL 접속
conn = pymssql.connect(host=r"211.233.21.82", user='bif', password='DWBIF2020!@#', database='DB_PRICE', charset='utf8')

# Connection 으로부터 Cursor 생성
cursor = conn.cursor()
try:
    for i in range(len(data)-1):
        #SQL문 실행
        cursor.execute("INSERT INTO dbo.price_table(date, product_ID,company, price)  VALUES (%s,%s,%s,%s);", (nowDate,data[i][0],data[i][1],data[i][3]))
except:
    easygui.msgbox('추가된 데이터 오류 혹은 중복 실행으로 인한 오류', '대원CTS 최저가 비교 크롤링')

else:   
    conn.commit()
    conn.close()

2020-11-23


# 디비로부터 데이터 추출

In [10]:

# MSSQL 접속
conn = pymssql.connect(host=r"211.233.21.82", user='bif', password='DWBIF2020!@#', database='DB_PRICE', charset='utf8')

# Connection 으로부터 Cursor 생성
cursor = conn.cursor()
 
# SQL문 실행
cursor.execute('SELECT * FROM dbo.price_table;')
row = cursor.fetchall()
cols = [item[0] for item in cursor.description]
df=pd.DataFrame(row,columns=cols)

df.head()

,date,product_id,price,company
0,2020-11-19,DM500S9Z-AD3BA,"548,720¿ø",SAMSUNG
1,2020-11-19,FA506IU-HN174,"1,199,000¿ø",ASUS
2,2020-11-19,GA502IV-AZ001T,"1,699,000¿ø",ASUS
3,2020-11-19,NT750QCJ-K58A,"1,797,990¿ø",SAMSUNG
4,2020-11-19,NT950XCJ-X716A,"2,087,700¿ø",SAMSUNG


In [11]:
import re

long=len(df['price'])

for i in range(long):
    slt=re.findall("[0-9]", df['price'][i])
    df['price'][i]=int(''.join(slt))
    
df.to_csv("C:\\Users\\user\\Desktop\\result.csv")

df

,date,product_id,price,company
0,2020-11-19,DM500S9Z-AD3BA,548720,SAMSUNG
1,2020-11-19,FA506IU-HN174,1199000,ASUS
2,2020-11-19,GA502IV-AZ001T,1699000,ASUS
3,2020-11-19,NT750QCJ-K58A,1797990,SAMSUNG
4,2020-11-19,NT950XCJ-X716A,2087700,SAMSUNG
...,...,...,...,...
106,2020-11-23,NT550XCR-AD1WA,498990,SAMSUNG
107,2020-11-23,NT550XCR-AD2A,598000,SAMSUNG
108,2020-11-23,NT550XCR-AD5A + NVMe SSD 512GB,998000,SAMSUNG
109,2020-11-23,NT550XCR-GD5A,1198990,SAMSUNG
